In [44]:
%pylab inline
import sys,os
sys.path.append("..")
import tempfile
import shutil
import trappy
trace_sched = "/home/leoy/Documents/Work_disk/tools/x86/trappy/tests/trace_sched.txt"

Populating the interactive namespace from numpy and matplotlib


In [45]:
def setup_sched():
    tDir = tempfile.mkdtemp(dir="/tmp", prefix="trappy_doc", suffix = ".tempDir")
    shutil.copyfile(trace_sched, os.path.join(tDir, "trace.txt"))
    return tDir

In [46]:
temp_sched_location = setup_sched()
run1 = trappy.Run(temp_sched_location)

In [47]:
run1.sched_load_avg_cpu.data_frame

,__comm,__cpu,__pid,cpu,load,utilization
Time,,,,,,
0.000002,sshd,0,2962,0,13,18


In [48]:
trappy.Run(temp_sched_location).sched_load_avg_task.data_frame

,__comm,__cpu,__pid,avg_period,comm,load,pid,runnable_avg_sum,running_avg_sum,utilization
Time,,,,,,,,,,
9.999994e-07,trace-cmd,4,2971,48595,sshd,0,2962,0,0,0


In [49]:
events = { 
    '1001.456789' : { 'task': 'rcu_preempt',       'pid': 1123, 'cpu': 001 },
    '1002.456789' : { 'task': 'rs:main',           'pid': 2123, 'cpu': 002 },
    '1003.456789' : { 'task': 'AsyncTask #1',      'pid': 3123, 'cpu': 003 },
    '1004.456789' : { 'task': 'kworker/1:1H',      'pid': 4123, 'cpu': 004 },
    '1005.456789' : { 'task': 'jbd2/sda2-8',       'pid': 5123, 'cpu': 005 },
}

in_data = """"""
for timestamp in sorted(events):
    in_data+="{0:>16s}-{1:d} [{2:04d}] {3:s}: event0:   tag=value\n".\
            format(
                events[timestamp]['task'],
                events[timestamp]['pid'],
                events[timestamp]['cpu'],
                timestamp
                )

with open("/tmp/trace1.txt", "w") as fout:
    fout.write(in_data)

In [50]:
trappy.register_dynamic('Event0', 'event0', scope="sched")
run3 = trappy.Run("/tmp/trace1.txt")
run3.event0.data_frame

,__comm,__cpu,__pid,tag
Time,,,,
0,rcu_preempt,1,1123,value
1,rs:main,2,2123,value
2,AsyncTask #1,3,3123,value
3,kworker/1:1H,4,4123,value
4,jbd2/sda2-8,5,5123,value


In [51]:
    l = trappy.ILinePlot(
            run1,                                            # TRAPpy run Object
            run1.sched_load_avg_cpu,                         # TRAPpy Event (maps to a unique word in the Trace)
            column=[                                         # Column(s)
                "load",
                "utilization"],
            
            filters={                                        # Filter the data  
                "cpu": [
                    0]},
            pivot="cpu",                                     # One plot for each pivot will be created
        
            per_line=1)                                      # Number of graphs per line
    l.view()

""
""
